# 1. Imports and Setup

In [ ]:
# Import required modules
import asyncio

from pylabrobot.resources import Coordinate
from pylabrobot.thermocycling.thermocycler import Thermocycler
from pylabrobot.thermocycling.backends.simulated import SimulatedThermocyclerBackend
from pylabrobot.thermocycling.standard import Step, Stage, Protocol


# 2. Instantiate Thermocycler

In [ ]:
# Instantiate a simulated backend (for demo/testing)
backend = SimulatedThermocyclerBackend()

# Create the thermocycler object
thermocycler = Thermocycler(
    name="demo_thermocycler",
    size_x=100.0,
    size_y=100.0,
    size_z=50.0,
    backend=backend,
    child_location=Coordinate(0, 0, 0)
)

# 3. Quickstart – Set and Read Temperatures

In [ ]:
async def quickstart_demo():
    # Open the lid
    await thermocycler.open_lid()

    # Set temperatures
    await thermocycler.set_lid_temperature([105.0])
    await thermocycler.set_block_temperature([95.0])

    # Wait for target temps to be reached
    await thermocycler.wait_for_lid()
    await thermocycler.wait_for_block()

    # Read and display temperatures
    lid_temp = await thermocycler.get_lid_current_temperature()
    block_temp = await thermocycler.get_block_current_temperature()
    print(f"Lid Temp: {lid_temp}")
    print(f"Block Temp: {block_temp}")

asyncio.run(quickstart_demo())


# 4. Run PCR Profile

In [ ]:
async def run_pcr_profile_demo():
    await thermocycler.run_pcr_profile(
        denaturation_temp=[95.0], denaturation_time=30,
        annealing_temp=[55.0], annealing_time=30,
        extension_temp=[72.0], extension_time=60,
        num_cycles=3,
        block_max_volume=50,
        lid_temperature=[105.0],
        pre_denaturation_temp=[95.0], pre_denaturation_time=60,
        final_extension_temp=[72.0], final_extension_time=90,
        storage_temp=[4.0], storage_time=600
    )

    # Wait for completion
    await thermocycler.wait_for_profile_completion()
    print("PCR protocol completed.")

asyncio.run(run_pcr_profile_demo())



## 5. Monitoring status

In [ ]:
async def monitor_status():
    print("Lid Status:", await thermocycler.get_lid_status())
    print("Block Status:", await thermocycler.get_block_status())

    print("Cycle Progress:",
          await thermocycler.get_current_cycle_index(),
          "/", await thermocycler.get_total_cycle_count())

    print("Step Progress:",
          await thermocycler.get_current_step_index(),
          "/", await thermocycler.get_total_step_count())

asyncio.run(monitor_status())

